# Logic Apps as MCP Server

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **© 2025 Ozgur Guler. All rights reserved.**

---

This notebook demonstrates connecting an Azure AI Foundry agent to a Logic Apps Standard workflow exposed as an MCP (Model Context Protocol) server.

## What is MCP?

MCP is an open standard that lets AI agents discover and invoke external tools. Think of it as **"USB-C for AI agents"** - a universal way for LLMs to access enterprise resources.

## Architecture

```
┌─────────────────┐     MCP Protocol      ┌──────────────────┐
│  Foundry Agent  │ ◄──────────────────► │  Logic Apps MCP  │
│   (MCP Client)  │                       │     Server       │
└─────────────────┘                       └────────┬─────────┘
                                                   │
                                          ┌────────▼─────────┐
                                          │  1,400+ Connectors│
                                          │  (SAP, Salesforce,│
                                          │   SharePoint...)  │
                                          └──────────────────┘
```

---

## Prerequisites

1. **Logic Apps Standard** app with MCP enabled
2. **Workflow** with Request trigger exposed as MCP tool
3. **MCP Server URL**: `https://<logic-app>.azurewebsites.net/api/mcpservers/<name>/mcp`

In [ ]:
!pip install azure-ai-projects azure-ai-agents azure-identity --pre --quiet

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

# Foundry Configuration
PROJECT_ENDPOINT = os.getenv(
    "PROJECT_ENDPOINT",
    "https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212"
)

# Logic Apps MCP Server Configuration
# Replace with your Logic Apps MCP endpoint
MCP_SERVER_URL = os.getenv(
    "MCP_SERVER_URL",
    "https://your-logic-app.azurewebsites.net/api/mcpservers/ticketing/mcp"
)
MCP_SERVER_LABEL = "logic-apps-ticketing"

MODEL = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-5-nano")

print(f"Project: {PROJECT_ENDPOINT}")
print(f"MCP Server: {MCP_SERVER_URL}")

---

## Create Agent with MCP Tool

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import McpTool

# Initialize client
client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# Configure MCP tool pointing to Logic Apps
mcp_tool = McpTool(
    server_label=MCP_SERVER_LABEL,
    server_url=MCP_SERVER_URL,
    allowed_tools=[],  # Empty = allow all tools
)

print(f"MCP Tool configured: {MCP_SERVER_LABEL}")

In [ ]:
# Create agent with MCP tool
agent = client.agents.create_agent(
    model=MODEL,
    name="logic-apps-mcp-agent",
    instructions="""You are a helpful assistant with access to enterprise tools via Logic Apps.
Use the available MCP tools to perform tasks like creating tickets, sending emails, etc.""",
    tools=mcp_tool.definitions,
)

print(f"Created agent: {agent.id}")

---

## Invoke Agent with MCP Tool

In [ ]:
import time
from azure.ai.agents.models import (
    ListSortOrder,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)

# Create thread and send message
thread = client.agents.threads.create()
message = client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a support ticket for 'Login issue on mobile app'",
)

# Run agent with MCP tool resources
run = client.agents.runs.create(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_resources=mcp_tool.resources,
)

print(f"Run started: {run.id}")

# Process run (handle tool approvals if needed)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    
    # Auto-approve MCP tool calls
    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        approvals = [
            ToolApproval(tool_call_id=tc.id, approve=True, headers=mcp_tool.headers)
            for tc in run.required_action.submit_tool_approval.tool_calls
            if isinstance(tc, RequiredMcpToolCall)
        ]
        if approvals:
            client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_approvals=approvals)
            print(f"Approved {len(approvals)} tool call(s)")

print(f"Run completed: {run.status}")

In [ ]:
# Get response
messages = client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    if msg.text_messages:
        print(f"{msg.role.upper()}: {msg.text_messages[-1].text.value}")

In [ ]:
# Cleanup
client.agents.delete_agent(agent.id)
print("Agent deleted")

---

## Setting Up Logic Apps as MCP Server

### 1. Enable MCP in `host.json`

```json
{
  "extensions": {
    "workflow": {
      "McpServerEndpoints": {
        "enable": true
      }
    }
  }
}
```

### 2. Define MCP Servers in `mcpservers.json`

```json
{
  "mcpServers": [
    {
      "name": "ticketing",
      "description": "IT Ticketing System",
      "tools": [
        { "name": "CreateTicket" },
        { "name": "CloseTicket" }
      ]
    }
  ]
}
```

### 3. MCP Endpoint URL

```
https://<logic-app-name>.azurewebsites.net/api/mcpservers/<server-name>/mcp
```

---

## Summary

| Step | Code |
|------|------|
| **1. Configure MCP Tool** | `McpTool(server_url=..., server_label=...)` |
| **2. Create Agent** | `client.agents.create_agent(tools=mcp_tool.definitions)` |
| **3. Run with Resources** | `client.agents.runs.create(tool_resources=mcp_tool.resources)` |
| **4. Handle Approvals** | `ToolApproval(tool_call_id=..., approve=True)` |

## Next

Continue to `../08-connectors-as-mcp-tools`.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>